# Energy Analysis

In this bonus notebook, we will explore the locality of a system for different energy envelopes.

Given a set of eigenvectors of the Hamiltonian, we can compute a corresponding density matrix by computing an outer product between those eigenvectors and those same eigenvectors transposed. For example, consider the matrices of the mini protein system.

In [ ]:
from scipy.io import mmread
hfile = "Matrices/hamiltonian_sparse.mtx"
ofile = "Matrices/overlap_sparse.mtx"
H = mmread(hfile).todense()
O = mmread(ofile).todense()

We'll work in the Lowdin basis.

In [ ]:
from scipy.linalg import eigh
from numpy import sqrt, diag
w, v= eigh(O)
for i in range(0, len(w)):
    w[i] = 1.0/sqrt(w[i])
isq = v.dot(diag(w)).dot(v.T)

In [ ]:
WH = isq.dot(H.dot(isq))

Now compute the eigenvalues of the matrix.

In [ ]:
from scipy.linalg import eigh
w, v = eigh(WH)

And finally the density matrix

In [ ]:
nel = 2612
D = v[:,:nel].dot(v[:,:nel].T)
print(D.shape)

We know that this matrix is sparse. In fact, let's compute the sparsity.

In [ ]:
def compute_size(mat, thresh=1e-5):
    c = 0
    for j in range(0, mat.shape[0]):
        for i in range(0, mat.shape[1]):
            if abs(mat[i,j]) > thresh:
                c += 1
    return (1.0*c)/(mat.shape[0]*mat.shape[1])

In [ ]:
print("Density Sparsity:", compute_size(D))

When we computed the density matrix, we choose to use all of the occupied orbitals. But in fact we could have choosen a different subset of orbitals, and computed a matrix of the exact same shape. What other subsets might be interesting? First, let's look at the spectrum:

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1,1,figsize=(8,4))
plt.plot(w, '.', markersize=2)

Interestingly enough, there are many gaps in this spectrum, not just the band gap. Let's define those sets:

In [ ]:
w_sets =[]
w_sets.append(list(range(0,nel)))
start = 0
# Just to a little past the gap because we can't trust BigDFT's basis functions beyond that
for i in range(1, nel+100):
    if w[i] - w[i-1] > 0.02:
        w_sets.append(list(range(start,i)))
        print(start,i)
        start = i

Let's also add an arbitrary subset as well for contrast

In [ ]:
w_sets.append([500, 800])

Lets look at the sparsity of those portions

In [ ]:
D_list = []
for s in w_sets:
    Ds = v[:,s].dot(v[:,s].T)
    D_list.append(Ds)
    print("Density Sparsity: (", s[0], s[-1], ")", compute_size(Ds))

These sub density matrices are sparse, indeed in some cases they are even more sparse than the real density matrix. This means we can analyze them with the same purity indicator technique we have been using for the full density matrix.

In [ ]:
geom_file = "Matrices/1L2Y.yaml"
from BigDFT import Fragments as F
from yaml import load
with open(geom_file) as ifile:
    sys = load(ifile)
    positions = sys["Reading positions"].itervalues().next()
fdict = F.CreateFragDict(positions)

I'll bring in the infastructure for computing the purity values.

In [ ]:
from numpy import zeros
electron_lookup = {'H' :1, 'He':2, 
                   'Li':1, 'Be':2, 'B' :3, 'C': 4, 'N':5, 'O':6, 'F' :7, 'Ne':8,
                   'Na':1, 'Mg':2, 'Al':3, 'Si':4, 'P':5, 'S':6, 'Cl':7}
natoms = len(positions["positions"])
charge = zeros((natoms))
for i, p in enumerate(positions["positions"]):
    name = p.keys()[1]
    charge[i] = electron_lookup[name]

In [ ]:
from copy import deepcopy
frag_lists = {}
lval = []
for fname in fdict.keys():
    if fname != "WAT":
        continue
    for fid in fdict[fname].keys():
        lval.append(fdict[fname][fid])
    frag_lists["WAT"] = deepcopy(lval)
lval = []
for fname in fdict.keys():
    if fname != "SOD":
        continue
    for fid in fdict[fname].keys():
        lval.append(fdict[fname][fid])
    frag_lists["SOD"] = deepcopy(lval)
lval = []
for fname in fdict.keys():
    if fname != "CLA":
        continue
    for fid in fdict[fname].keys():
        lval.append(fdict[fname][fid])
    frag_lists["CLA"] = deepcopy(lval)
lval = []
for fname in fdict.keys():
    if fname == "WAT" or fname == "SOD" or fname == "CLA":
        continue
    for fid in fdict[fname].keys():
        lval.append(fdict[fname][fid])
    frag_lists["PRO"] = deepcopy(lval)

In [ ]:
def compute_purity(Den, charge, frag):
    from numpy import zeros
    from numpy import trace
    from scipy.sparse import csr_matrix
    if (len(frag)) == 0:
        return 0
    indices = []
    cv = 0
    for atom in frag:
        indices += atom_to_basis[atom-1]
        cv += charge[atom-1]

    submat = Den[indices,:]
    submat = submat[:,indices]
    
    return 2*trace(submat.dot(submat) - submat)/cv

In [ ]:
from CheSS import Matrices as M
metadata_file = "Matrices/sparsematrix_metadata.dat"
alookup = M.get_atomic_lookup(metadata_file)
atom_to_basis = [[] for x in range(0, max(alookup)+1)]
for basis, atom in enumerate(alookup):
    atom_to_basis[atom].append(basis)

And compute for every matrix.

In [ ]:
from copy import deepcopy
energy_purity = {}

for frag_type in ["WAT", "SOD", "CLA", "PRO"]:
    purity_values = {}
    for s, den in zip(w_sets, D_list):
        pl = []
        for frag in frag_lists[frag_type]:
            pl.append(compute_purity(den, charge, frag))
            name = str(s[0])+":"+str(s[-1])
        purity_values[name] = deepcopy(pl)
    energy_purity[frag_type] = purity_values

In [ ]:
markers = [".", 'x', '<', '+']
marker_dict = {}
for m, k in zip(markers, energy_purity["WAT"].keys()):
    marker_dict[k] = m

fig, ax = plt.subplots(len(["WAT", "SOD", "CLA", "PRO"]), 1, figsize=(12,12))
ax[0].set_title("Purity Values")
ax[-1].set_xlabel("Fragment ID")

for i, frag_type in enumerate(["WAT", "SOD", "CLA", "PRO"]):
    for name, val in energy_purity[frag_type].items():
        ax[i].plot(val, 'x', markersize=12, label=name)
    ax[i].set_ylabel(frag_type)
    first = energy_purity[frag_type].keys()[0]
    ax[i].set_xlim(-1,len(energy_purity[frag_type][first]))
#     ax[i].set_ylim([-0.1, 0.01])
ax[0].legend(loc="best", bbox_to_anchor=(1, 1))